In [25]:
import numpy as np
import os
import random

In [319]:
# point_map = {}
class Agent():
    def __init__(self):
        self.points = 0
        self.cards = ['A'] + [str(i) for i in range(2,11)] + ['10']*3
        self.usable_ace = False
    def action(self):
        if self.points<20:
            # if random.random()>0.5:
            # print(random.randint(0,len(self.cards)))
            # print(self.cards)
            card_sampled = self.cards[np.random.randint(0,len(self.cards))]
            if card_sampled == 'A' and self.points + 11 <=21:
                self.points += 11
                self.usable_ace = True
            elif card_sampled=='A' and self.points + 11 > 21:
                self.points += 1
                self.usable_ace = False
            else:
                self.usable_ace = False
                self.points += int(card_sampled)
            return card_sampled
        else:
            return 'S'
    def check(self):
        if self.points <=21:
            return False
        return True
              

class Dealer():
    def __init__(self):
        self.points = 0
        self.cards = ['A'] + [str(i) for i in range(2,11)] + ['10']*3
        self.showing_card = None
    def action(self):
        if self.points<17:
            card_sampled = self.cards[np.random.randint(0,len(self.cards))]
            self.showing_card = card_sampled
            if card_sampled == 'A' and self.points + 11 <=21:
                self.points += 11
            
            elif card_sampled=='A' and self.points + 11 > 21:
                self.points += 1
            else:
                self.points += int(card_sampled)
            return card_sampled        
        else:
            return 'S'
        
    def check(self):
        if self.points >21:
            return 'B'
        elif self.points == 21:
            return 'W'
        else:
            return True
        
        
## continue coding the black jack env by simulating the moves for a fixed trajectory length *from 6:56 PM yesterday
# class dealer(agent):
class blackjack():
    def __init__(self,dealer,agent):
        # self.cards = ['A'] + [str(i) for i in range(2,11)] + ['10']*3
        self.game_state = None
        self.turn_flag = 0
        self.turn = ['A','D']
        
        self.dealer = dealer
        self.agent = agent

        self.players = {'D':self.dealer,'A':self.agent}

    def hit(self):
        return self.cards[np.random.randint(0,len(self.cards))]        
    # player cards sum, usable ace yes/no, dealer card showing

    def stick(self):
        self.turn_flag+=1
        self.turn_flag%=2
        return self.turn[self.turn_flag]
    
    def episode(self):
        T = 30
        t = 0
        self.ep = []
        # initial cards
        CD = self.players['D'].action()
        CA = self.players['A'].action()
        # print(CA,CD)
        self.ep.append([self.players['A'].points,
                self.players['D'].showing_card,
                self.players['A'].usable_ace])

        while t<T:
            t+=1
            
            # print('time step = ',t,'   A',self.players['A'].points,'D',self.players['D'].points)
            if self.players['A'].points == 21 and self.players['D'].points<21:
                # self.ep.append([self.players['A'].points,
                #             self.players['D'].showing_card,
                #             self.players['A'].usable_ace])
                # print('1')
                return +1
            
            if self.players['D'].points>21:
                # self.ep.append([self.players['A'].points,
                #                 self.players['D'].showing_card,
                #                 self.players['A'].usable_ace])

                # print('2')
                return +1

            elif self.players['D'].points==21 and self.players['A'].points==21:
                # self.ep.append([self.players['A'].points,
                #                 self.players['D'].showing_card,
                #                 self.players['A'].usable_ace])
                # print('3')      
                return 0

            elif self.players['D'].points==21 and self.players['A'].points<21:
                # self.ep.append([self.players['A'].points,
                #                 self.players['D'].showing_card,
                #                 self.players['A'].usable_ace])
                # print('4') 
                return -1

            if not (self.players['A'].check()):
                
                # first the player will move then the dealer 
                player = self.players[self.turn[self.turn_flag]]
                action = player.action()
                # print('action = ',action)
                if action == 'S':
                    self.stick()
                    self.ep.append([self.players['A'].points,
                                self.players['D'].showing_card,
                                self.players['A'].usable_ace])

                else:
                    self.ep.append([self.players['A'].points,
                                self.players['D'].showing_card,
                                self.players['A'].usable_ace])

            else:
                # self.ep.append([self.players['A'].points,
                #     self.players['D'].showing_card,
                #     self.players['A'].usable_ace])

                return -1

        # print('A',self.players['A'].points,'D',self.players['D'].points)
        
        # if exited the loop due to infinite sticking

        if (self.players['A'].points>self.players['D'].points):
            return +1
        elif self.players['A'].points==self.players['D'].points:
            return 0
        else:
            return -1
            


In [346]:
def first_visit_MC(state_list,final_reward,   returns = {},num_trials=100000):
    k = len(state_list)-1
    G = 0
    G += final_reward/num_trials
    while k>-1:
        state = state_list[k]
        # print(state)
        if '_'.join([str(state[i]) for i in range(0,len(state))]) not in returns:
            returns['_'.join([str(state[i]) for i in range(0,len(state))]) ] =0
        
        if state not in state[:k]:
            returns['_'.join([str(state[i]) for i in range(0,len(state))])] += G
        k-=1
    return returns

In [372]:
returns = {}
num_trials = 10000#100000
for _ in range(0,num_trials):
    random.seed()
    dealer = Dealer()
    agent = Agent()
    # print('p')
    b = blackjack(dealer,agent)
    

    final_reward = b.episode()
    episode = b.ep
    # print(episode)
    results = ['Draw','Win','Lose']
    returns = first_visit_MC(episode,final_reward,returns,num_trials)    


In [373]:
returns

{'23_10_False': -0.03369999999999994,
 '14_10_False': -0.023699999999999912,
 '12_10_False': -0.018499999999999944,
 '2_10_False': -0.009499999999999998,
 '25_9_False': -0.0067000000000000046,
 '15_9_False': -0.007900000000000008,
 '8_9_False': -0.0049,
 '6_9_False': -0.0035999999999999977,
 '27_A_False': -0.005200000000000001,
 '17_A_False': -0.008200000000000006,
 '7_A_False': -0.005800000000000002,
 '27_7_False': -0.0039999999999999975,
 '17_7_False': -0.004599999999999999,
 '9_7_False': -0.0039999999999999975,
 '28_7_False': -0.005700000000000002,
 '19_7_False': -0.007700000000000007,
 '16_7_False': -0.005600000000000002,
 '13_7_False': -0.004299999999999998,
 '10_7_False': -0.0025999999999999994,
 '20_10_False': 0.7372999999999351,
 '20_7_False': 0.26659999999998696,
 '20_6_False': 0.16659999999999797,
 '20_2_False': 0.15769999999999895,
 '10_2_False': -0.0024,
 '24_10_False': -0.030999999999999868,
 '17_10_False': -0.027999999999999886,
 '24_2_False': -0.008100000000000007,
 '18_

In [292]:
episode

[[9, '7', False],
 [15, '7', False],
 [16, '7', False],
 [18, '7', False],
 [28, '7', False]]

In [11]:
b.__dict__

{'cards': ['A', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', 10, 10, 10]}